In [1]:
import numpy as np
import os,sys
from PIL import Image, ImageOps, ImageChops
from original_helpers import *
from new_helpers import *
import math

%matplotlib inline

In [2]:
# GLOBAL VARIABLES

TOTAL_IMAGES = 50 # Number of images to load
TRAIN_FRACTION = 0.9 # Percentage of images used for training
ANGLE_STEP = 36 # Gotta be 360/ANGLE_STEP needs to be an integer
FLIP = True # Flag to signal if flipped  versions of rotated images should also be created
PATCH_SIZE = 16
PATCH_TRANSLATION = 8 # WARNING: this quickly explodes to enormous amount of data if small patch_translation is selected.
FOREGROUND_THRESHOLD = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
ORIGINAL_IMAGE_WIDTH = 400
ORIGINAL_IMAGE_HEIGHT = 400

In [3]:
# Loaded a set of images
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(TOTAL_IMAGES, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [img_float_to_uint8(load_image(image_dir + files[i])) for i in range(n)]

train_size = int(TOTAL_IMAGES*TRAIN_FRACTION)
train_imgs = imgs[0:train_size]
test_imgs = imgs[train_size:]
imgs_rotated_flipped = rotate_imgs(train_imgs, ANGLE_STEP, FLIP)
train_imgs += imgs_rotated_flipped
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [img_float_to_uint8(load_image(gt_dir + files[i])) for i in range(n)]
train_gt_imgs = gt_imgs[0:train_size]
test_gt_imgs = gt_imgs[train_size:]
gt_imgs_rotated_flipped = rotate_imgs(train_gt_imgs, ANGLE_STEP, FLIP)
train_gt_imgs += gt_imgs_rotated_flipped
print(files[0])

Loading 50 images
satImage_001.png
Loading 50 images
satImage_001.png


In [4]:
def patch_range(w, h, w_translation, h_translation):
    h_range = [0] if h_translation <= 0 else list(range(0, h, h_translation))
    minus_h_range = [ x * -1 for x in h_range]
    h_range = list(set(h_range + minus_h_range)) # trick to remove duplicates
    h_range.sort()
    
    w_range = [0] if w_translation <= 0 else list(range(0, h, h_translation))
    minus_w_range = [ x * -1 for x in w_range]
    w_range = list(set(w_range + minus_w_range)) # trick to remove duplicates
    w_range.sort()
    
    return h_range, w_range

def img_crop(img, w, h, w_translation, h_translation):
    list_patches = []
    imgwidth = img.shape[0]
    imgheight = img.shape[1]
    is_2d = len(img.shape) < 3
    if is_2d:
        pil_img = Image.fromarray(img, 'L')
    else:
        pil_img = Image.fromarray(img, 'RGB')
        
    h_range, w_range = patch_range(w, h, w_translation, h_translation)
    for it in h_range:
        for jt in w_range:
            
            for i in range(0,imgheight,h):
                for j in range(0,imgwidth,w):
                    im_patch = pil_img.crop((j + jt, i + it, j + jt + w, i + it + h))
                    list_patches.append(np.array(im_patch))
    return list_patches

def extract_patches(imgs, patch_size, patch_translation):
    img_patches = [img_crop(imgs[i], patch_size, patch_size, patch_translation, patch_translation) for i in range(len(imgs))]
    img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
    return img_patches

In [5]:
train_img_patches = extract_patches(train_imgs, PATCH_SIZE, PATCH_TRANSLATION)
train_gt_patches = extract_patches(train_gt_imgs, PATCH_SIZE, PATCH_TRANSLATION)
test_img_patches = extract_patches(test_imgs, PATCH_SIZE, PATCH_TRANSLATION)
test_gt_patches = extract_patches(test_gt_imgs, PATCH_SIZE, PATCH_TRANSLATION)

In [6]:
# Compute features for each image patch
X = np.asarray([ extract_features(train_img_patches[i]) for i in range(len(train_img_patches))])
Y = np.asarray([value_to_class(np.mean(train_gt_patches[i]), FOREGROUND_THRESHOLD) for i in range(len(train_gt_patches))])
Xi = np.asarray([ extract_features(test_img_patches[i]) for i in range(len(test_img_patches))])
Yi = np.asarray([value_to_class(np.mean(test_gt_patches[i]), FOREGROUND_THRESHOLD) for i in range(len(test_gt_patches))])

In [7]:
# Print feature statistics

print('Computed ' + str(X.shape[0]) + ' features')
print('Feature dimension = ' + str(X.shape[1]))
print('Number of classes = ' + str(np.max(Y)))

Y0 = [i for i, j in enumerate(Y) if j == 0]
Y1 = [i for i, j in enumerate(Y) if j == 1]
print('Class 0: ' + str(len(Y0)) + ' samples')
print('Class 1: ' + str(len(Y1)) + ' samples')

Computed 4809375 features
Feature dimension = 6
Number of classes = 1
Class 0: 3336085 samples
Class 1: 1473290 samples


In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)  
X_train = scaler.transform(X)
clf = MLPClassifier()# default options
clf.fit(X_train, Y)
clf.score(X_train,Y)
Xi = scaler.transform(Xi)  
Zi = clf.predict(Xi)

In [9]:
from IPython.core.debugger import Tracer


def unpatch(patches_array):
    h_patches, w_patches = patch_range(PATCH_SIZE, PATCH_SIZE, PATCH_TRANSLATION, PATCH_TRANSLATION)
    h_patches_rescaled = [x + abs(min(h_patches)) for x in h_patches]
    w_patches_rescaled = [x + abs(min(w_patches)) for x in w_patches]
    number_of_patches = (len(h_patches) * len(w_patches))
    
    number_of_pictures = patches_array.shape[0] / number_of_patches
    if not number_of_pictures.is_integer() :
        print('Something is wrong. Dimensions are incorrect!')
        return
    
    unpatched_imgs = []
    for img_no in range(int(number_of_pictures)):
        pil_patched_image = Image.new('L', (ORIGINAL_IMAGE_WIDTH + w_patches_rescaled[-1], ORIGINAL_IMAGE_HEIGHT + h_patches_rescaled[-1]))
        pil_index_image = pil_patched_image.copy()
        
        for j, h_patch in enumerate(h_patches_rescaled):
            for i, w_patch in enumerate(w_patches_rescaled):
                patch_mask = Image.new('L', (ORIGINAL_IMAGE_WIDTH + w_patches_rescaled[-1], ORIGINAL_IMAGE_HEIGHT + h_patches_rescaled[-1]))
                index_mask = patch_mask.copy()

                prediction_index = (img_no * number_of_patches) + j * len(h_patches_rescaled) + i
                patch = np.ones((PATCH_SIZE, PATCH_SIZE), dtype='uint8')

                prediction_patch = patch * patches_array[prediction_index]
                index_patch = patch

                pil_prediction_patch = Image.fromarray(prediction_patch, 'L')
                pil_index_patch = Image.fromarray(index_patch, 'L')

                upper_left_corner = (w_patch, h_patch)
                patch_mask.paste(pil_prediction_patch, upper_left_corner)
                index_mask.paste(pil_index_patch, upper_left_corner)

                pil_patched_image = ImageChops.add(pil_patched_image, patch_mask)
                pil_index_image = ImageChops.add(pil_index_image, index_mask)

        patched_image = np.array(pil_patched_image)
        index_image = np.array(pil_index_image)

        h_from = h_patches[-1]
        h_to = (ORIGINAL_IMAGE_HEIGHT + h_patches[-1])
        w_from = w_patches[-1]
        w_to = (ORIGINAL_IMAGE_WIDTH + w_patches[-1])
        unpatched_img = np.around((patched_image/index_image))[h_from:h_to, w_from:w_to]
        unpatched_imgs.append(unpatched_img)
    return unpatched_imgs

In [10]:
unpached_Zi = unpatch(Zi)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:48: RuntimeWarning: invalid value encountered in true_divide


In [11]:
len(unpached_Zi)

3125

In [12]:
unpached_Zi[0].shape

(400, 400)

In [13]:
unpached_Yi = unpatch(Yi)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:48: RuntimeWarning: invalid value encountered in true_divide


In [14]:
len(unpached_Zi)

3125

In [15]:
unpached_Yi[0].shape

(400, 400)

In [16]:
from sklearn.metrics import f1_score

f1_score(Yi, Zi, average='binary')

0.67168871226051363